# 首先声明一下，学习AIGC药物之前，请您需要先学习AIDD药物的知识，因为AIGC药物的分子表示方法与AIDD药物有很大不同。

# AIGC在药物设计中的应用：分子表示与机器学习模型

## 主要内容
- 1. 导入必要的库
- 2. 加载数据
- 3. 数据预处理
    - 3.1 转换SMILES为RDKit分子对象
    - 3.2 生成活性标签
    - 3.3 数据集划分
- 4. 分子表示生成
    - 4.1 分子指纹（Molecular Fingerprints）
    - 4.2 机器学习模型
        - 4.2.1 随机森林
        - 4.2.2 支持向量机
        - 4.2.3 卷积神经网络（CNN）
- 5. 模型评估
    - 5.1 训练集上的性能
    - 5.2 测试集上的性能
    - 5.3 绘制ROC曲线
- 6. 总结与展望

## 准备工作
- 1. 安装Anaconda
- 2. 安装RDKit
- 3. 安装PyTorch
- 4. 安装相关库

## 注意事项
- 1. 该教程基于Python3.9，Anaconda3，RDKit2020.03，PyTorch-GPU版本的。
- 2. 该教程基于`AIDD`数据集，该数据集包含了大量的AIDD药物的SMILES字符串和活性数据。
- 3. 该教程基于AIDD药物的活性数据，但也可以应用于AIGC药物的活性数据。
- 4. 该教程基于AIDD药物的SMILES字符串，但也可以应用于AIGC药物的SMILES字符串。
- 5. 该教程基于AIDD药物的分子表示生成，但也可以应用于AIGC药物的分子表示生成。

    

## 1. 导入必要的库

- **Jupyter Notebook** 用于演示如何通过`chembl.csv`中的SMILES生成分子表示，并应用多种机器学习模型进行药物活性预测。


In [1]:
# 检测是否安装了GPU
import torch
if torch.cuda.is_available():
    print('Using GPU')
else:
    print('Using CPU')

Using CPU


In [2]:
# 首先，我们需要导入所需的Python库。这些库包括数据处理、分子表示生成、机器学习模型以及可视化工具。
import pandas as pd # 数据处理
import numpy as np # 数值计算
import os # 文件和目录操作
import matplotlib.pyplot as plt # 可视化工具
import seaborn as sns # 可视化工具

from rdkit import Chem # 分子表示生成
from rdkit.Chem import AllChem, Descriptors # 分子表示生成

from sklearn.model_selection import train_test_split # 数据集划分
from sklearn.preprocessing import StandardScaler # 数据标准化
from sklearn.decomposition import PCA # 主成分分析
from sklearn.ensemble import RandomForestClassifier # 随机森林
from sklearn.svm import SVC # 支持向量机
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report # 评价指标

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# For CNN on SMILES sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

In [3]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)

/Users/wangyang/Desktop/AIGC-in-drug-design/01_Molecular_representation/data


## 2. 加载数据

In [4]:
# 定义数据路径
data_path = DATA / 'bioactivities_compounds_VEGFR2.csv'

# 加载数据
df = pd.read_csv(data_path)

# 显示数据的前几行
df.head()

,molecule_chembl_id,IC50,units,smiles,pIC50
0,CHEMBL5189340,0.023,nM,CN(C)c1ccc(/C=N/NC(=O)Cn2nc(Cc3ccc(Cl)cc3)c3cc...,10.638272
1,CHEMBL429743,0.030,nM,COc1cc2nccc(Oc3ccc4c(C(=O)Nc5ccc(Cl)cc5)cccc4c...,10.522879
2,CHEMBL5186748,0.120,nM,CNC(=O)c1cc(Oc2ccc(NC(=O)c3nn(-c4ccc(Cl)cc4)cc...,9.920819
3,CHEMBL3093581,0.140,nM,O=C(Nc1ccc(Oc2ccc3nc(NC(=O)C4CC4)cn3c2)c(F)c1)...,9.853872
4,CHEMBL3586072,0.140,nM,CNC(=O)c1ccc(-c2ccc(NC(=O)Nc3cc(Br)cc(C(F)(F)F...,9.853872


## 3. 数据预处理
### 3.1 转换SMILES为RDKit分子对象
- 我们需要将SMILES字符串转换为RDKit分子对象，以便生成分子表示。

In [5]:
# 转换SMILES为RDKit分子对象
df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))

# 移除无效的SMILES
df = df[df['mol'].notnull()].reset_index(drop=True)

print(f"有效分子数量: {len(df)}")


有效分子数量: 200


### 3.2 生成活性标签
- 如果数据集中存在pic50列，我们将基于该列生成二分类标签：pIC50 >= 6.3为活性阳性（1），否则为阴性（0）。如果没有pic50列，则进行随机划分。

In [6]:
# 检查是否存在pic50列
if 'pIC50' in df.columns:
    # 生成二分类标签，❤️这里我做 实验 ，估计减少了数据，采用的8.8作为阈值，可以自己调整
    df['active'] = df['pIC50'].apply(lambda x: 1 if x >= 8.9 else 0)
    print("使用 pic50 划分活性标签")
else:
    # 如果没有pic50列，创建一个随机标签
    df['active'] = np.random.randint(0, 2, size=len(df))
    print("没有 pic50 列，使用随机标签")


使用 pic50 划分活性标签


### 3.3 数据集划分
- 根据是否存在pic50列，选择不同的划分方式。


In [7]:
df.head()

,molecule_chembl_id,IC50,units,smiles,pIC50,mol,active
0,CHEMBL5189340,0.023,nM,CN(C)c1ccc(/C=N/NC(=O)Cn2nc(Cc3ccc(Cl)cc3)c3cc...,10.638272,<rdkit.Chem.rdchem.Mol object at 0x7fb11b05d3c0>,1
1,CHEMBL429743,0.030,nM,COc1cc2nccc(Oc3ccc4c(C(=O)Nc5ccc(Cl)cc5)cccc4c...,10.522879,<rdkit.Chem.rdchem.Mol object at 0x7fb11b05d120>,1
2,CHEMBL5186748,0.120,nM,CNC(=O)c1cc(Oc2ccc(NC(=O)c3nn(-c4ccc(Cl)cc4)cc...,9.920819,<rdkit.Chem.rdchem.Mol object at 0x7fb11b05d4a0>,1
3,CHEMBL3093581,0.140,nM,O=C(Nc1ccc(Oc2ccc3nc(NC(=O)C4CC4)cn3c2)c(F)c1)...,9.853872,<rdkit.Chem.rdchem.Mol object at 0x7fb11b05d510>,1
4,CHEMBL3586072,0.140,nM,CNC(=O)c1ccc(-c2ccc(NC(=O)Nc3cc(Br)cc(C(F)(F)F...,9.853872,<rdkit.Chem.rdchem.Mol object at 0x7fb11b05d580>,1


In [18]:
# 特征和标签

# if 'pIC50' in df.columns:
#     # 这个是有pic50列的情况
#     X   = df['smiles']
#     y   = df['active']
#     # 划分训练集和测试集df['active'] 如果是0，则为测试集，1为训练集
#     X_train = df[df['active'] == 1]['smiles']
#     X_test = df[df['active'] == 0]['smiles']
#     y_train = df[df['active'] == 1]['active']
#     y_test = df[df['active'] == 0]['active']
# 
#     print(f"训练集大小: {len(X_train)}")
#     print(f"测试集大小: {len(X_test)}")
# else:
# 这个是没有pic50列的情况
X = df['smiles']
y = df['active']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y if 'pIC50' in df.columns else None
)

print(f"训练集大小: {len(X_train)}")
print(f"测试集大小: {len(X_test)}")


训练集大小: 160
测试集大小: 40


## 4. 分子表示生成

### 4.1 分子指纹（Molecular Fingerprints）

使用RDKit生成Morgan指纹（也称为ECFP4指纹）。

Morgan指纹是一种基于圆筛的分子指纹，它可以捕捉到分子的相似性和排斥性。

说明：
- `radius`：指纹半径，即生成的指纹包含的原子数量。
- `n_bits`：指纹长度，即生成的指纹的二进制位数。
- `useFeatures=True`：是否使用特征来生成指纹。
- `useChirality=True`：是否考虑化学键来生成指纹。
- `useBondTypes=True`：是否考虑化学键类型来生成指纹。
- `useCounts=True`：是否考虑原子计数来生成指纹。

下面的get_morgan_fingerprint函数使用RDKit的AllChem模块来生成Morgan指纹。

In [19]:
def get_morgan_fingerprint(smiles, radius=2, n_bits=2048):
    """
    Generate Morgan fingerprints for a SMILES string.
    Args:
        smiles (str): SMILES string.
        radius (int): Morgan fingerprint radius.
        n_bits (int): Number of bits in the fingerprint.
    Returns:
        np.array: Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros(n_bits)

# 生成指纹
X_train_fingerprints = np.array([get_morgan_fingerprint(smile) for smile in X_train])
X_test_fingerprints = np.array([get_morgan_fingerprint(smile) for smile in X_test])

print("分子指纹生成完成")


分子指纹生成完成


### 4.2 图表示（Graph Representation）

利用RDKit生成基于图的分子描述符，例如分子指纹的不同类型、分子图的拓扑特征等。这里我们将使用分子描述符作为图表示的简化版本。

分子描述符是指根据分子的结构和化学特性，计算得到的特征向量。
说明：
- `rdkit.Chem.rdchem.Mol`：RDKit分子对象。
- `rdkit.Chem.rdchem.Mol.GetAtoms()`：获取分子的所有原子。
- `rdkit.Chem.rdchem.Mol.GetBonds()`：获取分子的所有化学键。
- `rdkit.Chem.rdchem.Bond.GetBondType()`：获取化学键的类型。
- `rdkit.Chem.rdchem.Bond.GetBeginAtomIdx()`：获取化学键的起始原子索引。
- `rdkit.Chem.rdchem.Bond.GetEndAtomIdx()`：获取化学键的终止原子索引。
- `rdkit.Chem.rdchem.Atom.GetAtomicNum()`：获取原子的原子编号。
- `rdkit.Chem.rdchem.Atom.GetTotalDegree()`：获取原子的总度数。
- `rdkit.Chem.rdchem.Atom.GetTotalValence()`：获取原子的总价电数。
- `rdkit.Chem.rdchem.Atom.GetTotalNumHs()`：获取原子的总氢原子。

下面的get_mol_descriptors函数使用RDKit的Descriptors模块来生成分子描述符。

In [ ]:
def get_molecular_descriptors(mol):
    """
    Generate molecular descriptors for a RDKit molecule object.
    Args:
        mol (rdkit.Chem.rdchem.Mol): RDKit molecule object.
    Returns:
        dict: Molecular descriptors.
    """
    descriptors = {}
    for desc_name, func in Descriptors.descList:
        descriptors[desc_name] = func(mol)
    return descriptors

# 生成分子描述符
X_train_graph = df.loc[X_train.index, 'mol'].apply(get_molecular_descriptors).apply(pd.Series).fillna(0).values
X_test_graph = df.loc[X_test.index, 'mol'].apply(get_molecular_descriptors).apply(pd.Series).fillna(0).values

print("图表示生成完成")


### 4.3 连续表示（Continuous Representation）

使用PCA将指纹降维至较低的维度，以获得连续表示。
说明：
- `n_components`：降维后的维度。
- `random_state`：随机数种子。

下面的get_continuous_representation函数使用scikit-learn的PCA模块来降维。

In [ ]:
# 标准化指纹
scaler = StandardScaler()
X_train_fingerprints_scaled = scaler.fit_transform(X_train_fingerprints)
X_test_fingerprints_scaled = scaler.transform(X_test_fingerprints)

# PCA降维
pca = PCA(n_components=100, random_state=RANDOM_STATE)
X_train_continuous = pca.fit_transform(X_train_fingerprints_scaled)
X_test_continuous = pca.transform(X_test_fingerprints_scaled)

print("连续表示生成完成")


## 5. 机器学习模型训练与评估

### 5.1 随机森林（Random Forest）
随机森林是一种基于树的集成学习方法，它可以有效地处理高维、非线性和缺失数据。
说明：
- `n_estimators`：树的数量。
- `random_state`：随机数种子。
公式：
- $y = \frac{1}{N} \sum_{i=1}^N \sum_{c=1}^C \left[ \frac{w_c}{N_c} \sum_{j:x_j \in R_c(x_i)} y_j \right]$
- $R_c(x_i)$：第$c$类样本中与$x_i$距离最近的样本集合。
- $N_c$：第$c$类样本的数量。
- $w_c$：第$c$类样本的权重。



下面的train_random_forest函数使用scikit-learn的RandomForestClassifier模块来训练随机森林模型。

In [ ]:
# 训练随机森林模型
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf.fit(X_train_fingerprints, y_train)

# 预测概率
y_pred_rf_proba = rf.predict_proba(X_test_fingerprints)[:, 1]


5.2 支持向量机（SVM）
支持向量机（SVM）是一种二类分类模型，它通过最大化间隔边界来间隔数据点。
说明：
- `C`：软间隔惩罚参数。
- `kernel`：核函数类型。
- `gamma`：核函数参数。
公式：
- $f(x) = \sum_{i=1}^N \alpha_i y_i K(x_i, x) + b$
- $K(x_i, x_j) = \left\{\begin{matrix} \exp(-\gamma ||x_i - x_j||^2) & \text{if } x_i \neq x_j \\ 0 & \text{if } x_i = x_j \end{matrix}\right.$
- $\alpha_i$：第$i$个支持向量的拉格朗日乘子。
- $b$：偏置项。

下面的train_svm函数使用scikit-learn的SVC模块来训练SVM模型。

In [ ]:
# 训练SVM模型
svm = SVC(probability=True, random_state=RANDOM_STATE)
svm.fit(X_train_fingerprints, y_train)

# 预测概率
y_pred_svm_proba = svm.predict_proba(X_test_fingerprints)[:, 1]


### 5.3 人工神经网络（ANN）

构建一个简单的全连接神经网络。

说明：
    - `input_dim`：输入特征的维度。
    - `hidden_dim`：隐藏层的维度。
    - `output_dim`：输出层的维度。
    - `num_layers`：隐藏层的数量。
    - `dropout`：dropout概率。
    - `learning_rate`：学习率。
    - `num_epochs`：训练轮数。

公式：
    - $f(x) = \sigma(W_2 \sigma(W_1 x + b_1) + b_2)$
    - $\sigma(x) = \frac{1}{1 + e^{-x}}$    
    - $W_1, b_1, W_2, b_2$：权重和偏置。

下面的train_ann函数使用PyTorch的nn模块来构建ANN模型。

In [ ]:
class ANN(nn.Module):
    '''
    ANN模型: 输入层 -> 隐藏层 -> 输出层
    '''
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.relu1(self.fc1(x))
        out = self.relu2(self.fc2(out))
        out = self.sigmoid(self.fc3(out))
        return out

# 创建数据集
class MoleculeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 训练与测试数据集
train_dataset = MoleculeDataset(X_train_continuous, y_train)
test_dataset = MoleculeDataset(X_test_continuous, y_test)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 初始化模型
ann = ANN(input_size=X_train_continuous.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(ann.parameters(), lr=0.001)

# 训练模型
epochs = 20 # 训练轮数
ann.train()
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = ann(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 评估模型
ann.eval()
y_pred_ann_proba = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = ann(X_batch)
        y_pred_ann_proba.extend(outputs.numpy().flatten())

print("ANN训练与预测完成")


### 5.4 卷积神经网络（CNN）

构建一个简单的1D卷积神经网络。

说明：
    - `input_size`：输入特征的维度。
    - `num_classes`：输出层的维度。
    - `num_filters`：卷积核的数量。
    - `kernel_size`：卷积核的大小。
    - `padding`：填充的大小。
    - `pool_size`：池化层的大小。
    - `num_epochs`：训练轮数。

公式：
    - $f(x) = \sigma(W_2 \sigma(W_1 x + b_1) + b_2)$
    - $\sigma(x) = \frac{1}{1 + e^{-x}}$    
    - $W_1, b_1, W_2, b_2$：权重和偏置。

这里，我们将使用SMILES字符串的字符级嵌入并应用1D CNN进行预测。

In [ ]:
# 使用CountVectorizer将SMILES转化为特征向量
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3))
X_train_smiles = vectorizer.fit_transform(X_train).toarray()
X_test_smiles = vectorizer.transform(X_test).toarray()

# 标准化
scaler_cnn = StandardScaler()
X_train_smiles_scaled = scaler_cnn.fit_transform(X_train_smiles)
X_test_smiles_scaled = scaler_cnn.transform(X_test_smiles)

# 定义CNN模型
class SimpleCNN(nn.Module):
    def __init__(self, input_size, num_classes=1):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(64 * (input_size // 4), 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.pool1(self.relu1(self.conv1(x)))
        out = self.pool2(self.relu2(self.conv2(out)))
        out = out.view(out.size(0), -1)
        out = self.relu3(self.fc1(out))
        out = self.sigmoid(self.fc2(out))
        return out

# 创建数据集
class SMILESDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset_cnn = SMILESDataset(X_train_smiles_scaled, y_train)
test_dataset_cnn = SMILESDataset(X_test_smiles_scaled, y_test)

train_loader_cnn = DataLoader(train_dataset_cnn, batch_size=32, shuffle=True)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=32, shuffle=False)

# 初始化模型
input_size_cnn = X_train_smiles_scaled.shape[1]
cnn = SimpleCNN(input_size=input_size_cnn)
criterion_cnn = nn.BCELoss()
optimizer_cnn = optim.Adam(cnn.parameters(), lr=0.001)

# 训练模型
epochs_cnn = 10
cnn.train()
for epoch in range(epochs_cnn):
    for X_batch, y_batch in train_loader_cnn:
        optimizer_cnn.zero_grad()
        outputs = cnn(X_batch)
        loss = criterion_cnn(outputs, y_batch)
        loss.backward()
        optimizer_cnn.step()
    print(f"Epoch [{epoch+1}/{epochs_cnn}], Loss: {loss.item():.4f}")

# 评估模型
cnn.eval()
y_pred_cnn_proba = []
with torch.no_grad():
    for X_batch, _ in test_loader_cnn:
        outputs = cnn(X_batch)
        y_pred_cnn_proba.extend(outputs.numpy().flatten())

print("CNN训练与预测完成")


## 6. 模型评估与可视化
### 6.1 评估指标

我们将使用AUC-ROC曲线来评估模型的性能。

AUC-ROC曲线：
- 横轴：真正例率（TPR）= 真阳性率（TPR）= 召回率（Recall）= TPR = TP / (TP + FN)
- 纵轴：假正例率（FPR）= 假阴性率（FPR）= 1 - 特异性（Specificity）= FPR = FP / (FP + TN)
- 面积：AUC = 0.5 * (TPR + FPR)

AUC值：
- 0.5：随机模型
- 1：完美模型
- 0：最差模型

### 6.2 ROC曲线

ROC曲线（Receiver Operating Characteristic Curve）：
- 横轴：FPR
- 纵轴：TPR
- 面积：AUC

AUC值：
- 0.5：随机模型
- 1：完美模型
- 0：最差模型

下面的plot_roc函数使用scikit-learn的roc_curve函数来绘制ROC曲线。

In [ ]:
# 计算ROC曲线和AUC
# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf_proba)
roc_auc_rf = auc(fpr_rf, tpr_rf)

# SVM
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_pred_svm_proba)
roc_auc_svm = auc(fpr_svm, tpr_svm)

# ANN
fpr_ann, tpr_ann, _ = roc_curve(y_test, y_pred_ann_proba)
roc_auc_ann = auc(fpr_ann, tpr_ann)

# CNN
fpr_cnn, tpr_cnn, _ = roc_curve(y_test, y_pred_cnn_proba)
roc_auc_cnn = auc(fpr_cnn, tpr_cnn)

# 绘制ROC曲线
plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, color='blue', label=f'RF (AUC = {roc_auc_rf:.2f})')
plt.plot(fpr_svm, tpr_svm, color='green', label=f'SVM (AUC = {roc_auc_svm:.2f})')
plt.plot(fpr_ann, tpr_ann, color='red', label=f'ANN (AUC = {roc_auc_ann:.2f})')
plt.plot(fpr_cnn, tpr_cnn, color='purple', label=f'CNN (AUC = {roc_auc_cnn:.2f})')
plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.grid(True)
# 保存ROC曲线，DPI为300
plt.savefig(DATA/'ROC_curve.png', dpi=300)
plt.show()

## 7. 结论

通过上述步骤，我们成功地：

1. 生成了分子指纹、图表示和连续表示。
2. 训练了随机森林、支持向量机、人工神经网络和卷积神经网络模型。
3. 评估了各模型的性能，并通过ROC曲线和AUC值进行可视化。

这些模型在药物活性预测中展示了不同的性能特点，帮助研究人员选择合适的工具进行药物设计与优化。